# Stats used in Table 1 (ablation study)

In [1]:
!pip install --quiet bootstrapped

In [2]:
import os
import numpy as np
import re
from glob import glob
import yaml
import pickle
import pandas as pd
import bootstrapped.bootstrap as bs
import bootstrapped.stats_functions as bs_stats

In [3]:
def read_data(dirname, strategy):
    eval_file = os.path.join(dirname, "eval.pkl")
    config_file = os.path.join(dirname, ".hydra", "config.yaml")
    with open(config_file, "r") as f:
        config = yaml.safe_load(f)
    with open(eval_file, "rb") as f:
        data = pickle.load(f)

    # read setting
    arr_setting = [strategy]
    rows = []
    header = None
    for i, (ins, res, info) in enumerate(data):
        if header is None:
            header = ["strategy", "ins", "num_agents"] 
            header = header + list(res.get_dict_wo_paths().keys()) + list(info.keys())
        rows.append(arr_setting + [i, ins.num_agents] + list(res.get_dict_wo_paths().values()) + list(info.values()))
    return pd.DataFrame(rows, columns=header)

In [4]:
DATADIR = "/data/exp/ctrm_sampling_ablation/"
subdirs = [re.split("/", x)[-2] for x in sorted(glob(f"{DATADIR}/*/stats.txt"))]
df_normal = read_data(os.path.join(DATADIR, subdirs[0]), "normal")
df_wo_comm = read_data(os.path.join(DATADIR, subdirs[1]), "wo_comm")
df_wo_ind = read_data(os.path.join(DATADIR, subdirs[2]), "wo_ind")
df_wo_random_walk = read_data(os.path.join(DATADIR, subdirs[3]), "wo-random-walk")
df = pd.concat([
    df_normal,
    df_wo_comm,
    df_wo_ind,
    df_wo_random_walk,
])
df

,strategy,ins,num_agents,solved,name_planner,elapsed_planner,sum_of_costs,maximum_costs,sum_of_travel_dists,maximum_travel_dists,...,elapsed_static_collide,elapsed_continuous_collide,lowlevel_expanded,lowlevel_explored,sample_nums,elapsed_roadmap_gen,cnt_roadmap_static_collide,cnt_roadmap_continuous_collide,elapsed_roadmap_static_collide,elapsed_roadmap_continuous_collide
0,normal,0,26,True,PrioritizedPlanning,1.974766,651,45,17.294870,1.212742,...,0,0.868702,20191,19328,34213,18.639103,0,217502,0,4.744596
1,normal,1,28,False,PrioritizedPlanning,2.275688,0,0,0.000000,0.000000,...,0,0.933907,22921,22044,40526,22.531860,0,280552,0,6.179521
2,normal,2,21,True,PrioritizedPlanning,0.821060,465,37,12.348527,0.915848,...,0,0.347281,11016,10038,24657,14.126421,0,155137,0,3.242114
3,normal,3,29,False,PrioritizedPlanning,0.912079,0,0,0.000000,0.000000,...,0,0.344473,13553,12968,37809,20.498436,0,250332,0,5.537666
4,normal,4,21,True,PrioritizedPlanning,1.185470,401,46,10.546270,1.157597,...,0,0.491761,14593,13845,22426,12.520246,0,156681,0,3.286375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,wo-random-walk,95,27,False,PrioritizedPlanning,0.337087,0,0,0.000000,0.000000,...,0,0.112782,6159,5718,15883,17.719028,0,352897,0,7.326213
96,wo-random-walk,96,26,False,PrioritizedPlanning,0.079015,0,0,0.000000,0.000000,...,0,0.019799,2090,1938,23148,25.874945,0,531510,0,11.219489
97,wo-random-walk,97,22,True,PrioritizedPlanning,0.517046,393,28,10.435886,0.852654,...,0,0.229062,5983,5588,7791,10.592732,0,189139,0,3.853403
98,wo-random-walk,98,28,False,PrioritizedPlanning,0.330615,0,0,0.000000,0.000000,...,0,0.112918,5365,4993,17783,21.113544,0,426915,0,8.833138


In [5]:
# without wo_rw
df = pd.concat([
    df_normal,
    df_wo_comm,
    df_wo_ind,
])
df

num_strategies = len(df.groupby(["strategy"]))
df_sub = df.query(f"solved == 1")
print("**success rate**")
display(df_sub.groupby("strategy")["solved"].count())
print()
all_success_indexes = tuple(df_sub.groupby("ins")["name_planner"].count().loc[lambda x: x >= num_strategies].index)
df_all_success = df_sub.query(f"ins in {all_success_indexes}")
print(f"instances succeeded over all strategies: {len(all_success_indexes)} / 100\n")

print("\n**sum-of-costs**")
for strategy, res in df_all_success.groupby(["strategy"]):
    samples = np.array(res["sum_of_costs"] / res["num_agents"])
    print(strategy, bs.bootstrap(samples, stat_func=bs_stats.mean))
    
print("\n**expanded nodes**")
for strategy, res in df_all_success.groupby(["strategy"]):
    cnt = np.array(res["lowlevel_expanded"] / res["num_agents"])
    print(strategy, bs.bootstrap(cnt, stat_func=bs_stats.mean))

**success rate**


strategy
normal     80
wo_comm    23
wo_ind     33
Name: solved, dtype: int64


instances succeeded over all strategies: 11 / 100


**sum-of-costs**
normal 21.18157740517424    (20.30553577969388, 22.025172819275586)
wo_comm 28.716047389881382    (27.32915657026329, 30.154130145582716)
wo_ind 31.30547407357684    (30.46151849420525, 32.181907627908615)

**expanded nodes**
normal 612.4446598204898    (547.3429779826381, 673.2939294309482)
wo_comm 996.3455523761847    (922.6321863650917, 1067.2240916189285)
wo_ind 1058.5522267580172    (993.0623678600532, 1118.0910902054866)
